# Common

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm  # !pip install tqdm
from tqdm import trange
import multiprocessing as mp
import shutil
import glob
import os
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [2]:
!mkdir -p Temp

In [3]:
def bracket_row(row):    
    s = row['data']
    index = min(s.find('.'), s.find('('))
    data = row['data']
    row['data'] = data[0:index]
    row['bracket'] = data[index:]
    return row

In [4]:
def adjust(text,n=7):
    text = str(text)    
    return " " * (n - len(text)) + text

In [5]:
def bracket_to_ct(tag, data, bracket, deltaG, negative_deltaG=True):    
    deltaG = deltaG.replace('(','').replace(')','')
    deltaG = float(deltaG)
    if(deltaG > 0 and negative_deltaG ): # negetive?!
        deltaG = -1 * deltaG
    stack = []
    index = np.zeros((len(bracket)), dtype = int)
    values = np.zeros((len(bracket)), dtype = int)
    for i in range(len(bracket)):
        index[i] = i + 1
        if(bracket[i] == '.'):
            values[i] = 0
        elif(bracket[i] == '('):
            stack.append(i)
        elif(bracket[i] == ')'):
            if(len(stack) == 0 ):
                print('structure error!')
            values[stack[-1]] = i + 1
            values[i]  = stack[-1] + 1
            stack.pop()
        else:
            print('structure error!')
    if(len(stack) != 0 ):
        print('structure error!')
    # body    
    ct = f"{adjust(len(data),6)} dG ={adjust(deltaG,10)} {tag}\n"   
    for i in range(len(bracket)):
        ct += f"{adjust(index[i],6)} {data[i]} {adjust(i,6)} {adjust((i+2)%(len(data)+1),6)} {adjust(values[i],6)} {adjust(index[i],7)}\n"
    return ct

In [6]:
def fasta_to_df(path):
    with open(path, 'r') as file:
        text = file.read()
    lines = [line for line in text.split('\n') if len(line) > 0]
    s = ''
    tags = []
    data = []
    for l in lines:
        if(l[0]=='>'):
            tags.append(l)        
            data.append(s)
            s = ''
        else:
            s += l    
    data.append(s)
    df = pd.DataFrame(
            {
                'tag': tags,
                'data': data[1:]
            })
    df['tag'] = df['tag'].apply(lambda x: x[1:])    
    return df

In [7]:
def df_to_fasta(df, path):
    lines = []
    df.apply(lambda row: lines.append(f">{row['tag']}\n{row['data']}\n"),axis=1)
    with open(path,'w') as file:
        file.write(''.join(lines))

In [8]:
def reformat(path):
    return path.replace('(','_').replace(')','_').replace('.','').replace(':','_')

In [9]:
def reformatCT(path):
    with open(path, 'r') as file:
        text = file.read()
    text = [l for l in text.split('\n') if len(l) > 0 ] # remove blank lines
    text = '\n'.join(text)
    text = text.replace("\t"," ")
    while("  " in text):
        text = text.replace("  ", " ")
    lines = [l for l in text.split('\n')]
    for i in range(len(lines)):
        if(lines[i][0] == " "):
            lines[i] = lines[i][1:]
        if(lines[i][-1] == " "):
            lines[i] = lines[i][:-1]
    text = '\n'.join(lines)
    return text

In [10]:
def get_ct_data(ct):
    ct = "\n".join(ct.split('\n')[1:])
    df = pd.read_csv(StringIO(ct), sep=" ", header=None)               
    nucleotide = df.iloc[:,1]
    index = df.iloc[:,5]
    values = df.iloc[:,4]
    return [nucleotide, index, values]

In [11]:
def ct2dot_bracket(path):
    [nucleotide, index, values] = get_ct_data(reformatCT(path))
    text = ''.join(nucleotide) + "\n"
    watch = []
    for i, v in zip(index,values):
        if(v == 0):
            text += '.'
        else:
            if( v not in watch):
                text += '('
                watch.append(i)
            if( v in watch):
                text += ')'
    return text

In [12]:
def is_nested(index, values):
    max_value = max(index) + 10 # inf
    for i, v in zip(index, values):
        if(v < max_value and v != 0):
            max_value  = v
        if(i >= max_value):
            max_value = max(index) + 10 # inf
        if(v > max_value):
            return False               
    return True

In [13]:
'''ct = reformatCT('./secondary_structure/spot_rna/AMWY020598281_2832-3256_+_/AMWY020598281_2832-3256_+_.ct')
[nucleotide, index, values] = get_ct_data(ct)
print(is_nested( index,  values))
''';

### rename tag of input genome to new tag id

# Download dataset

In [13]:
'''
from Bio import Entrez
Entrez.email = "abolhasani.eliya@gmail.com"     
with Entrez.esearch(db='nucleotide', term="Arabidopsis thaliana") as handle:
    result = Entrez.read(handle)

print(result)
genome_ids = result['IdList']

for genome_id in genome_ids:
    print(genome_id)
    record = Entrez.efetch(db="nucleotide", id=genome_id, rettype="fasta", retmode="text")        
    with open(f'{genome_id}.fasta', 'w') as f:
        f.write(record.read())
    break
''';
'''
from Bio import Entrez
Entrez.email = "abolhasani.eliya@gmail.com"     
record = Entrez.efetch(db="nucleotide", id="NC_054143.4", rettype="fasta", retmode="text")        
with open(f'data.fasta', 'w') as f:
    f.write(record.read())
''';

In [14]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/439/995/GCA_000439995.3_AzaInd2.1/GCA_000439995.3_AzaInd2.1_genomic.fna.gz

--2021-11-14 18:10:04--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/439/995/GCA_000439995.3_AzaInd2.1/GCA_000439995.3_AzaInd2.1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.7, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85647577 (82M) [application/x-gzip]
Saving to: ‘GCA_000439995.3_AzaInd2.1_genomic.fna.gz’

CA_000439995.3_AzaI  27%[====>               ]  22.82M  7.26MB/s    eta 8s     ^C


In [263]:
!gzip -d ./GCA_000439995.3_AzaInd2.1_genomic.fna.gz

# Download data from Mirbase

In [28]:
directory = 'miRBase_driven_data'

In [18]:
base = "https://www.mirbase.org/ftp/CURRENT"        
!rm -r {directory}
!mkdir -p {directory}
!wget {base}/aliases.txt.gz -P ./{directory}/       ; gzip -d ./{directory}/aliases.txt.gz 
!wget {base}/hairpin.fa.gz -P ./{directory}/           ; gzip -d ./{directory}/hairpin.fa.gz 
!wget {base}/hairpin_high_conf.fa.gz -P ./{directory}/ ; gzip -d ./{directory}/hairpin_high_conf.fa.gz 
!wget {base}/mature.fa.gz -P ./{directory}/            ; gzip -d ./{directory}/mature.fa.gz 
!wget {base}/mature_high_conf.fa.gz -P ./{directory}/  ; gzip -d ./{directory}/mature_high_conf.fa.gz
!wget {base}/miRNA.str.gz -P ./{directory}/            ; gzip -d ./{directory}/miRNA.str.gz 
!wget {base}/miRNA.xls.gz -P ./{directory}/            ; gzip -d ./{directory}/miRNA.xls.gz 
!wget {base}/organisms.txt.gz -P ./{directory}/        ; gzip -d ./{directory}/organisms.txt.gz

--2021-11-14 19:08:31--  https://www.mirbase.org/ftp/CURRENT/aliases.txt.gz
Resolving www.mirbase.org (www.mirbase.org)... 130.88.97.249
Connecting to www.mirbase.org (www.mirbase.org)|130.88.97.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480536 (469K) [application/x-gzip]
Saving to: ‘./miRBase_driven_data/aliases.txt.gz’

aliases.txt.gz      100%[===================>] 469.27K   469KB/s    in 1.0s    

2021-11-14 19:08:32 (469 KB/s) - ‘./miRBase_driven_data/aliases.txt.gz’ saved [480536/480536]

--2021-11-14 19:08:33--  https://www.mirbase.org/ftp/CURRENT/hairpin.fa.gz
Resolving www.mirbase.org (www.mirbase.org)... 130.88.97.249
Connecting to www.mirbase.org (www.mirbase.org)|130.88.97.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1547350 (1.5M) [application/x-gzip]
Saving to: ‘./miRBase_driven_data/hairpin.fa.gz’

hairpin.fa.gz       100%[===================>]   1.48M   995KB/s    in 1.5s    

2021-11-14 19:08:34 (995 K

In [29]:
df = fasta_to_df(f'./{directory}/mature.fa')
#df = fasta_to_df('./Data/mature_high_conf.fa')
df['organism'] = df['tag'].apply(lambda x: x[:3])
print(df.shape)
df.head(2)

(48885, 3)


,tag,data,organism
0,cel-let-7-5p MIMAT0000001 Caenorhabditis elega...,UGAGGUAGUAGGUUGUAUAGUU,cel
1,cel-let-7-3p MIMAT0015091 Caenorhabditis elega...,CUAUGCAAUUUUCUACCUUACC,cel


In [30]:
organism = pd.read_csv(f'./{directory}/organisms.txt',sep='\t')
organism.columns = [c.replace('#','') for c in organism.columns] # remove sharp from columns
print(organism.shape)
organism.head(2)

(285, 5)


,organism,division,name,tree,NCBI-taxid
0,aqu,AQU,Amphimedon queenslandica,Metazoa;Porifera;,400682
1,nve,NVE,Nematostella vectensis,Metazoa;Cnidaria;,45351


In [31]:
items = list(organism['tree'].unique())
items.sort(key=len)
items

['Viruses;',
 'Mycetozoa;',
 'Alveolata;',
 'Metazoa;Porifera;',
 'Metazoa;Cnidaria;',
 'Viridiplantae;Chlorophyta;',
 'Viridiplantae;Embryophyta;',
 'Viridiplantae;Coniferophyta;',
 'Viridiplantae;Magnoliophyta;',
 'Metazoa;Bilateria;Deuterostoma;',
 'Chromalveolata;Heterokontophyta;',
 'Metazoa;Bilateria;Ecdysozoa;Nematoda;',
 'Metazoa;Bilateria;Lophotrochozoa;Annelida;',
 'Metazoa;Bilateria;Lophotrochozoa;Nemertea;',
 'Metazoa;Bilateria;Lophotrochozoa;Mollusca;',
 'Viridiplantae;Magnoliophyta;monocotyledons;',
 'Metazoa;Bilateria;Deuterostoma;Hemichordata;',
 'Metazoa;Bilateria;Deuterostoma;Echinodermata;',
 'Metazoa;Bilateria;Lophotrochozoa;Brachiopoda;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Hexapoda;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Crustacea;',
 'Metazoa;Bilateria;Lophotrochozoa;Platyhelminthes;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Chelicerata;',
 'Metazoa;Bilateria;Ecdysozoa;Arthropoda;Mandibulata;',
 'Viridiplantae;Magnoliophyta;eudicotyledons;Poaceae;',
 'M

In [32]:
selectedTree = organism[organism['tree'].apply(lambda x: "Viridiplantae;" in x)]
print(selectedTree.shape)
selectedTree.head(5)

(86, 5)


,organism,division,name,tree,NCBI-taxid
66,cre,CRE,Chlamydomonas reinhardtii,Viridiplantae;Chlorophyta;,3055
67,pta,PTA,Pinus taeda,Viridiplantae;Coniferophyta;,3352
68,ppt,PPT,Physcomitrella patens,Viridiplantae;Embryophyta;,3218
69,smo,SMO,Selaginella moellendorffii,Viridiplantae;Embryophyta;,88036
70,ath,ATH,Arabidopsis thaliana,Viridiplantae;Magnoliophyta;eudicotyledons;Bra...,3702


In [33]:
selected = df[df['organism'].isin(selectedTree['organism'])]
print(selected.shape)
selected.head()

(10414, 3)


,tag,data,organism
316,ath-miR156a-5p MIMAT0000166 Arabidopsis thalia...,UGACAGAAGAGAGUGAGCAC,ath
317,ath-miR156a-3p MIMAT0031865 Arabidopsis thalia...,GCUCACUGCUCUUUCUGUCAGA,ath
318,ath-miR156b-5p MIMAT0000167 Arabidopsis thalia...,UGACAGAAGAGAGUGAGCAC,ath
319,ath-miR156b-3p MIMAT0031866 Arabidopsis thalia...,UGCUCACCUCUCUUUCUGUCAGU,ath
320,ath-miR156c-5p MIMAT0000168 Arabidopsis thalia...,UGACAGAAGAGAGUGAGCAC,ath


In [34]:
df_to_fasta(selected,'./Temp/mature_microRNA_queries.fasta')

# Remove redundant

## cdhit-est

In [41]:
!cdhit/cd-hit-est -i ./Temp/mature_microRNA_queries.fasta  -o ./Temp/NR_mature_microRNA_queries.fasta \
    -c 1 -r 0 -G 1 -g 1 -b 30 -l 10 -aL 0 -AL 99999999 -aS 0 \
    -AS 99999999 -s 0 -S 0 

Program: CD-HIT, V4.8.1 (+OpenMP), Oct 23 2021, 21:45:39
Command: cdhit/cd-hit-est -i
         ./Temp/mature_microRNA_queries.fasta -o
         ./Temp/NR_mature_microRNA_queries.fasta -c 1 -r 0 -G 1
         -g 1 -b 30 -l 10 -aL 0 -AL 99999999 -aS 0 -AS 99999999
         -s 0 -S 0

Started: Sun Nov 14 19:23:14 2021
                            Output                              
----------------------------------------------------------------
total seq: 10414
longest and shortest : 28 and 17
Total letters: 222978
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 1M
Buffer          : 1 X 12M = 12M
Table           : 1 X 16M = 16M
Miscellaneous   : 0M
Total           : 30M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 96149440

comparing sequences from          0  to      10414
..........    10000  finished       5817  clusters

    10414  finished       6028  clusters

Approximate

## reformat

In [42]:
with open('./Temp/NR_mature_microRNA_queries.fasta.clstr','r') as file:
    text = file.read()
lines = [line for line in text.split('\n') if len(line) > 0]
cluster = []
seqid = []
last_cluster = ""
for l in lines:
    if(l[0]=='>'):        
        last_cluster = l.replace('>Cluster ',"C")
    else:        
        cluster.append(last_cluster)
        seqid.append(l.split(', >')[1].split('...')[0])                
seq2cluster = pd.DataFrame({'seqid': seqid,'cluster': cluster})
print(seq2cluster.shape)
seq2cluster.head(2)    

(10414, 2)


,seqid,cluster
0,cst-miR11332,C0
1,stu-miR7994b-5p,C1


In [43]:
df = fasta_to_df("./Temp/mature_microRNA_queries.fasta")
df['accession'] = df['tag'].apply(lambda x : x.split(' ')[0])
seq2cluster = pd.merge(df,seq2cluster,how="inner",left_on='accession',right_on="seqid")[['cluster','seqid','tag']]
print(seq2cluster.shape)
display(seq2cluster.head(2))
seq2cluster.to_csv('./Temp/seq2cluster.csv',index=False)

(10414, 3)


,cluster,seqid,tag
0,C5495,ath-miR156a-5p,ath-miR156a-5p MIMAT0000166 Arabidopsis thalia...
1,C1199,ath-miR156a-3p,ath-miR156a-3p MIMAT0031865 Arabidopsis thalia...


In [44]:
# todo: sorted first by cluster then by seqid
seq2cluster.sort_values("cluster").head(2)

,cluster,seqid,tag
9422,C0,cst-miR11332,cst-miR11332 MIMAT0044622 Cucumis sativus miR1...
7002,C1,stu-miR7994b-5p,stu-miR7994b-5p MIMAT0031188 Solanum tuberosum...


In [45]:
df = fasta_to_df("./Temp/NR_mature_microRNA_queries.fasta")
df['tag'] = df['tag'].apply(lambda x : x.split(' ')[0])
df = pd.merge(df,seq2cluster,how="inner",left_on='tag',right_on="seqid")[['cluster','data']]

lines = []
df.apply(lambda row: lines.append(f">{row['cluster']}\n{row['data']}\n"),axis=1)
print(df.shape)
with open('./Temp/BLASTn_queries.fasta','w') as file:
    file.write(''.join(lines))

(6028, 2)


# BlastN

!sudo apt-get install ncbi-blast+


In [46]:
!makeblastdb -in input_genome.fna \
             -dbtype nucl \
             -out ./Temp/blastn_database



Building a new DB, current time: 11/14/2021 19:23:46
New DB name:   /home/jupyter/plant_microRNA_prediction/Temp/blastn_database
New DB title:  input_genome.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 126142 sequences in 9.84965 seconds.


In [17]:
header = 'qseqid sseqid qstart qend sstart send qseq sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos frames qframe sframe sstrand qcovs qcovhsp qlen slen'

In [18]:
!blastn -query ./Temp/BLASTn_queries.fasta \
        -out ./Temp/BLASTn_result \
        -num_threads {mp.cpu_count()} \
        -db ./Temp/blastn_database \
        -word_size 7 \
        -penalty -3 \
        -reward 2 \
        -gapopen 5 \
        -gapextend 2 \
        -outfmt '6 qseqid sseqid qstart qend sstart send qseq sseq evalue bitscore score length pident nident mismatch positive gapopen gaps ppos frames qframe sframe sstrand qcovs qcovhsp qlen slen'       

In [19]:
df_blastn = pd.read_csv('./Temp/BLASTn_result', sep='\t',header=None)
df_blastn.columns = header.replace("  "," ").split(" ")
print(df_blastn.shape)
df_blastn.head(2)

(326849, 27)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,gaps,ppos,frames,qframe,sframe,sstrand,qcovs,qcovhsp,qlen,slen
0,C5495,AMWY02099822.1,1,20,1769,1750,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.004,37.4,...,0,100.0,1/-1,1,-1,minus,100,100,20,3308
1,C5495,AMWY02082313.1,1,20,5954,5973,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.004,37.4,...,0,100.0,1/1,1,1,plus,100,100,20,8471


In [20]:
threshold = 4
df_blastn['Nonconformity'] = df_blastn['qlen'] - (abs(df_blastn['qend'] - df_blastn['qstart']) + 1) + df_blastn['gaps'] + df_blastn['mismatch']
df_blastn = df_blastn[df_blastn['Nonconformity'] <= threshold]
print(df_blastn.shape)
df_blastn.head(2)

(80217, 28)


,qseqid,sseqid,qstart,qend,sstart,send,qseq,sseq,evalue,bitscore,...,ppos,frames,qframe,sframe,sstrand,qcovs,qcovhsp,qlen,slen,Nonconformity
0,C5495,AMWY02099822.1,1,20,1769,1750,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.004,37.4,...,100.0,1/-1,1,-1,minus,100,100,20,3308,0
1,C5495,AMWY02082313.1,1,20,5954,5973,TGACAGAAGAGAGTGAGCAC,TGACAGAAGAGAGTGAGCAC,0.004,37.4,...,100.0,1/1,1,1,plus,100,100,20,8471,0


In [21]:
# remore redundancy and hold best one base of Nonconformity value
df_blastn = df_blastn.sort_values(["Nonconformity", "evalue"], ascending = (True, True))
df_blastn = df_blastn.drop_duplicates(subset=['sseqid','sstart', 'send','sstrand'], keep='first')
df_blastn.to_csv('./Temp/filtered_out_blastn.csv')
print(df_blastn.shape)

(66445, 28)


# Result of the blastn to bed file

In [22]:
flanking_value = 200
df = df_blastn[['sseqid', 'sstart', 'send', 'sstrand','slen']]
df['ones'] = 1

<ipython-input-22-d52c72306552>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ones'] = 1


In [23]:
def switch(row):
    if(row['sstart'] > row['send']):        
        temp = row['sstart']
        row['sstart'] = row['send']
        row['send'] = temp
    return row
df = df.apply(lambda row: switch(row), axis=1)

In [24]:
def convert(inp):
    if(inp == "plus"):
        return "forward"
    if(inp == "minus"):
        return "reverse"
    raise Exception('Error, sstrand contains illegal word! only "plus" and "minus" are allowed')
df['strand'] = df['sstrand'].apply(lambda x: convert(x))

In [25]:
def convert2sign(inp):
    if(inp == "plus"):
        return "+"
    if(inp == "minus"):
        return "-"
    raise Exception('Error, sstrand contains illegal word! only "plus" and "minus" are allowed')
df['sign'] = df['sstrand'].apply(lambda x: convert2sign(x))

In [26]:
df['hit_length'] = df.apply(lambda row: abs(row['send'] - row['sstart']) + 1 ,axis=1)

## convert sstart and send from location to index (range)

In [27]:
df['sstart'] = df['sstart'].apply(lambda x: x - 1)

In [28]:
df['downstream_flanking'] = df['sstart'].apply(lambda x:  flanking_value if x > flanking_value else x)

In [29]:
df['upstream_flanking'] = df.apply(lambda row:  flanking_value if (row['send']+flanking_value) <= row['slen'] else row['slen'] - row['send'],axis=1)

In [30]:
df['hit_start'] = df.apply(lambda row: row['downstream_flanking'] if row['sign'] == "+" else row['upstream_flanking'],axis=1)

In [31]:
df['hit_end'] = df.apply(lambda row: row['downstream_flanking'] + row['hit_length'] if row['sign'] == "+" else row['upstream_flanking'] + row['hit_length'],axis=1)

In [32]:
df['sstart'] = df['sstart'].apply(lambda x: max(x - flanking_value, 0))
df['send'] = df.apply(lambda row: min(row['send'] + flanking_value , row['slen']),axis=1)

In [33]:
df['tag'] = df.apply(lambda row: f">{row['sseqid']}:{row['sstart']}-{row['send']}({row['sign']})",axis=1)
df['reformated_tag'] = df['tag'].apply(lambda t: reformat(t))
df[['tag', 'reformated_tag', 'hit_start', 'hit_end']].to_csv('./Temp/hit_index_info.csv')#, index=False)

In [34]:
df['location_tag'] = df.apply(lambda row: f">{row['sseqid']}|{row['sign']}|{row['sstart'] + 1}-{row['send']}|{row['hit_start']+1}-{row['hit_end']}",axis=1)
df[['location_tag']].to_csv('./Temp/pipe_seprated_location_list.csv',index=False)

In [35]:
df[['sseqid','sstart','send','strand','ones', 'sign']].to_csv('./Temp/extension_index.bed', 
        index=False, header=False, sep="\t")

# Extention


In [36]:
# !sudo apt-get install bedtools

In [37]:
!bedtools getfasta -fi ./input_genome.fna -fo ./Temp/extended_original.txt -s -bed ./Temp/extension_index.bed
!rm input_genome.fna.fai

index file ./input_genome.fna.fai not found, generating...


In [34]:
# todo: remove duplicated
'''
df = fasta_to_df("./Temp/extended.txt")
df = df.drop_duplicates(subset=['tag'], keep='first')
df_to_fasta(df,"./Temp/extended.txt")
len(df['tag'].unique())
''';

# Convert hit region to upper case and other region to lower case

In [38]:
ext = fasta_to_df('./Temp/extended_original.txt')
info = pd.read_csv('./Temp/hit_index_info.csv')
info['tag'] = info['tag'].apply(lambda x: x[1:])
print(info.shape)
info.head(2)

(66445, 5)


,Unnamed: 0,tag,reformated_tag,hit_start,hit_end
0,132836,AMWY02059828.1:2832-3256(+),>AMWY020598281_2832-3256_+_,200,224
1,300170,AMWY02004761.1:1853-2277(+),>AMWY020047611_1853-2277_+_,200,224


In [39]:
ext = ext.sort_values(by=['tag']).reset_index()
ext['help_tag'] = ext.apply(lambda r: r['tag']+ str(r.name),axis=1)
del ext['tag']

info = info.sort_values(by=['tag']).reset_index()
info['help_tag'] = info.apply(lambda row: row['tag']+ str(row.name),axis=1)
def redefined_tag(row):
    tag = row['tag']
    [sstart, send] = tag.split(':')[-1].split('(')[0].split('-')
    sstart = int(sstart) + 1
    sign = tag.split('(')[-1].split(')')[0]    
    return f"{tag.split(':')[0]}|{sstart}-{send}|{row['hit_start']+1}-{row['hit_end']}|{sign}"
info['tag'] = info.apply(lambda row: redefined_tag(row),axis=1)
ext = pd.merge(ext,info,how='inner', on='help_tag')

def emphasis_hit(row):
    seq = list(row['data'].lower())            
    s = row['hit_start']
    e = row['hit_end']
    seq[s:e] = list(''.join(seq[s:e]).upper())    
    return ''.join(seq)
    
ext['data'] = ext.apply(lambda row: emphasis_hit(row),axis=1)
df_to_fasta(ext[['tag','data']],"./Temp/extended_modified.txt")

# Extended validation

In [ ]:
df_blastn['hit'] = df_blastn['sseq'].apply(lambda x: x.replace('-', ''))
info = pd.read_csv('./Temp/hit_index_info.csv')
ext = fasta_to_df('./Temp/extended2.txt')

counter = 0
for index in df_blastn.index:
    hit = df_blastn['hit'][index]
    row = info[info['Unnamed: 0']== index].reset_index()
    tag = row['tag'][0][1:]
    hs = row['hit_start'][0]
    he = row['hit_end'][0]        
    tag = tag.replace('(+)',f"|{hs}-{he}(+)")
    tag = tag.replace('(-)',f"|{hs}-{he}(-)")            
    seq = ext[ext['tag']==tag]['data'].iloc[0]
    seq = ext[ext['tag']==tag]['data'].iloc[0]    
    if(seq[hs:he] != hit):
        print(tag, df_blastn['slen'][index])
        print(seq[hs:he])
        print(hit)
        print('\n\n')
        counter += 1                

In [178]:
df = pd.read_csv('check1.csv')
df.columns = ['tag', 'data']
def do(x):
    x = x[1:]
    x = x.split('|')
    s = int(x[2].split('-')[0]) - 1
    e = x[2].split('-')[1]
    return f"{x[0]}:{s}-{e}({x[1]})"
df['tag'] = df['tag'].apply(lambda x: do(x))
df.head(2)

,tag,data
0,AMWY02000003.1:4084-4506(+),ATACAATTGTCACATagtttacattattaatttccgcttaatttat...
1,AMWY02000003.1:4391-4810(+),ttctaattgcaaatttatgttatatttttaaatagaaaggGAGATT...


In [176]:
for t in df['tag']:
    check_seq = df[df['tag']==t]['data'].iloc[0]
    ext_seq = ext[ext['tag']==t]['data'].iloc[0]
    if(check_seq != ext_seq):
        print('error')        

In [167]:
ext = fasta_to_df('./Temp/extended.txt')
ext.head(2)

,tag,data
0,AMWY02059828.1:2832-3256(+),AAAGAATCAGCAATGGAAAAATAACCGGTTCTTAATTCAGcataac...
1,AMWY02004761.1:1853-2277(+),actaataatgCATGGCCATATATATCAAATCTACCATATgccattt...


# RNA 2d prediction

## Mfold

In [11]:
'''
# installation
!wget http://www.unafold.org/download/mfold-3.6.tar.gz
!tar -xvf ./mfold-3.6.tar.gz; rm ./mfold-3.6.tar.gz
%cd ./mfold-3.6
!./configure
!make
!make install
%cd ..
!sudo apt install texlive-font-utils
''';

In [12]:
#todo : add all hyperparameter(options) to GUI

In [110]:
counter = 0
base = "./secondary_structure/mfold/"
!rm -r {base}
!mkdir -p {base}
df = fasta_to_df('./Temp/extended_modified.txt')

for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)            
    with open(base + f"{tag}/SEQ.FASTA",'w') as file:
        file.write(f">{row['tag']}\n{row['data']}")
    counter += 1    
    if(counter >= 1000):
        break

In [111]:
remove_lock = False
def run_mfold(tag):
    tag = reformat(tag)
    %cd {base + tag}
    !mfold  SEQ="SEQ.FASTA" T=20     
    if(not remove_lock):
        !find . -not -name "*.ct" -not -name "*.pdf" -not -name "*SEQ.FASTA" -not -type d -delete
    %cd ../../..

if __name__ == '__main__':        
    pool = mp.Pool(mp.cpu_count())  
    pool.map(run_mfold, df['tag'].iloc[:1000])

/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000301|687-1105|201-219|-/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020001671|2603-3023|201-221|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000031|4085-4506|201-222|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000501|377-795|201-219|-/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020001481|2932-3190|201-218|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002101|3109-3527|201-219|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020001921|3070-3487|201-218|-/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002411|5665-6082|201-218|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002961|1232-1655|201-224|+/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002851|8518-8936|201-219|-/home/jupyter

15	140,10,20,30,16	5	2	16	150,15	40,50,60,4	14	14	3160,70,80,90,16		170,180,100,110,6	17	190,8	1	120,130,16	18	2	15	8	6	16	140,150,200,16	1	160,17	16	210,6	3	5	17	170,15	220,230,15	4	180,190,240,9	1	200,17	7	250,2	19	3	16	210,260,17	
Structure plots generated.
All done.
220,230,270,17	7	240,2	280,290,17	1	4	7	250,300,6	
Structure plots generated.
All done.
16	18	260,270,16	310,280,5	18	320,330,/home/jupyter/plant_microRNA_prediction2	3	
290,1	340,350,8	/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002411|8269-8690|201-222|+
4	300,310,320,17	/home/jupyter/plant_microRNA_prediction360,370,380,8	
18	18	mfold version 3.6
REUSE= NO
330,/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020001481|2932-3190|201-221|+8	
390,400,410, 
End of Fill
3	18	2	340,350,360,5	Save file created using nafold.
Minimum folding energy is -159.90 kcal/mol.
Energy increment is 8.00 kcal/mol.
SEQ.pnt created.
Sequence length is 422
17	17	mfold version 3.6
H-num file

10	/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002741|1881-2301|201-221|+
Structure plots generated.
All done.


RNA free energy files (version 2.3) at 20 degrees created.
10	Suboptimal foldings created.
11	17	16	13	14	Energy dot plot created.
10	mfold version 3.6
10,20,30,18	4	40,50,60,70,80,15	REUSE= NO
8	90,100,9	/home/jupyter/plant_microRNA_prediction14	
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002881|1667-2086|201-220|+1	110,120,SEQ.pnt created.
Sequence length is 421

8	19	5	11	11	3	RNA free energy files (version 2.3) at 20 degrees created.17	130,140,
Structure plots generated.
All done.
11	
10,mfold version 3.6
150,160,18	14	170,20,30,40,50,60,REUSE= NO
11	12	180,SEQ.pnt created.
5	70,80,90,15	190,Sequence length is 420
19	
Structure plots generated.
All done.
200,100,110,16	/home/jupyter/plant_microRNA_predictionRNA free energy files (version 2.3) at 20 degrees created.

/home/jupyter/plant_microRNA_prediction/seconda

8	
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020002891|2-420|201-219|-350,360,60,70,80,6	RNA free energy files (version 2.3) at 20 degrees created.
190,200,
2	15	Energy dot plot created.
10	370,380,390,13	90,100,10,20,210,16	/home/jupyter/plant_microRNA_predictionmfold version 3.6
400,410, 
End of Fill
Save file created using nafold.

/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020003341|4703-5044|201-221|+
30,40,50,60,70,110,120,Minimum folding energy is -143.50 kcal/mol.
Energy increment is 7.17 kcal/mol.
220,80,11	7	7	23	2	90,100,REUSE= NO
130,140,230,240,mfold version 3.6
H-num file created from plot file.
8	7	110,120,
Structure plots generated.
All done.
REUSE= NO
9	SEQ.pnt created.
Sequence length is 419
150,160,1	250,260,130,SEQ.pnt created.
Sequence length is 342
1,2,3,4,RNA free energy files (version 2.3) at 20 degrees created.

Structure plots generated.
All done.
170,180,140,270,3	5,6,7,8,9,9	24	14	190,200,210,10,20,30,RN

10,11,12,13,14,1,2,3,4,14	240,5	80,90,100,RNA free energy files (version 2.3) at 20 degrees created.
Suboptimal foldings created.
Energy dot plot created.
1	12	15,16,17,18,19,20,1	
Structure plots generated.
All done.
11	10,11,12,13,5,6,7,8,250,110,/home/jupyter/plant_microRNA_prediction10	21,22,23,
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000511|6024-6442|201-219|-10,20,30,40,50,60,
260,14,15,16, 
9,10,11,12,120,6	24,25,270,130,140,15	13, 
70,80,90,/home/jupyter/plant_microRNA_prediction26,27,28,29,mfold version 3.6

150,/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020000381|375-652|57-78|-100,110,120,130,280,
REUSE= NO
30, 
1	160,140,5	290,300,/home/jupyter/plant_microRNA_prediction
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020003061|6941-7355|201-215|+7	26	
1	mfold version 3.6
150,160,170,310,13	SEQ.pnt created.
Sequence length is 419
7	15	6	180,320,330,REUSE= NO
170,180,20	RNA free energy files (ve

...

9	16	260,270,
Structure plots generated.
All done.
5	280,H-num file created from plot file.
14	mfold version 3.6
REUSE= NO
Suboptimal foldings created.
Energy dot plot created.
10	15	290,1,2,3,SEQ.pnt created.
Sequence length is 419
Suboptimal foldings created.
Energy dot plot created.
19	300,310,8	4,5,6,7,8,9,RNA free energy files (version 2.3) at 20 degrees created.
10	9	320,10,11,12,13,14,15,16,6	10,20,30,40,1	/home/jupyter/plant_microRNA_prediction
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020014111|619-1037|201-219|+
4	330,340,350,17,18,19,50,60,70,360,370,20,21,22,23,24,80,90,100,17	17	mfold version 3.6
REUSE= NO

Structure plots generated.
All done.
380,390,400,410, 
End of Fill
6	25,26,27,28,29,110,120,SEQ.pnt created.
Sequence length is 419
Save file created using nafold.
Minimum folding energy is -102.50 kcal/mol.
Energy increment is 5.12 kcal/mol.
30, 
130,RNA free energy files (version 2.3) at 20 degrees created.
15	11	140,150,9	10,20,30,40,H-num

Suboptimal foldings created.
Energy dot plot created.
260,mfold version 3.6
RNA free energy files (version 2.3) at 20 degrees created.
10	21	30	270,REUSE= NO
10,20,30,40,50,16	14	280,SEQ.pnt created.
Sequence length is 382
60,70,80,290,300,RNA free energy files (version 2.3) at 20 degrees created.
10,20,90,100,9	22	19	310,30,40,50,60,17	3	110,120,320,330,8	1	3	70,80,90,130,140,340,350,100,150,360,370,380,390,15	22	3	4	110,120,160,170,400,410, 
End of Fill
Save file created using nafold.
11	130,180,190,17	Minimum folding energy is -111.90 kcal/mol.
Energy increment is 5.59 kcal/mol.
15	140,200,
Structure plots generated.
All done.
150,23	18	H-num file created from plot file.
1,2,1	4	210,160,9	3,4,5,6,7,20	220,230,170,4	8,9,10,11,12,240,180,5	16	13,14,15,16,250,190,18	23	4	17,18,19,20,260,/home/jupyter/plant_microRNA_prediction
200,210,16	/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020015211|2950-3368|201-219|-12	
21,22, 
270,280,290,mfold version 3.6
REUSE= NO


1	250,260,260,Sequence length is 419
280,13	2,3,4,5,6,7,8,270,290,300,RNA free energy files (version 2.3) at 20 degrees created.
10,20,30,40,270,280,9	9,10,11,12,13,14,280,290,50,60,70,80,310,320,290,300,14	15,16,17,18,19,300,90,100,110,330,340,350,360,310,320,14	20,21, 
310,120,130,330,340,350,/home/jupyter/plant_microRNA_prediction
370,380,390,400,410, 
End of Fill
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020014161|1-332|201-217|-
320,330,140,150,160,360,370,380,390,Save file created using nafold.
Minimum folding energy is -95.80 kcal/mol.
Energy increment is 4.79 kcal/mol.
7	4	340,350,170,mfold version 3.6
REUSE= NO
400,410, 
End of Fill
Save file created using nafold.
Minimum folding energy is -124.70 kcal/mol.
Energy increment is 6.24 kcal/mol.
2	14	14	360,370,380,180,190,SEQ.pnt created.
Sequence length is 332
H-num file created from plot file.
1,2,
Structure plots generated.
8	All done.
390,400,410, 
End of Fill
Save file created using nafold.
200,H-

10	370,380,390,400,410, 
End of Fill
240,180,Save file created using nafold.
Minimum folding energy is -115.50 kcal/mol.
Energy increment is 5.78 kcal/mol.
250,22	190,200,260,7	4	210,H-num file created from plot file.
1,2,270,7	220,11	3,4,5,6,7,8,13	280,8	16	11	230,240,250,9,10,11,12,13,290,260,270,14,15,300,310,11	280,290,300,310,320, 
320,330,330, 
End of Fill
Save file created using nafold.
Minimum folding energy is -99.90 kcal/mol.
Energy increment is 5.00 kcal/mol.
340,350,8	23	H-num file created from plot file.
360,370,380,390,8	5	12	12	1,2,3,4,400,410, 
End of Fill
Save file created using nafold.
Minimum folding energy is -131.10 kcal/mol.
Energy increment is 6.55 kcal/mol.
9	5,6,7,8,9,10, 
14	17	Suboptimal foldings created.
Energy dot plot created.
H-num file created from plot file.
10	1,2,3,4,5,6,7,12	6	8,9,10,11,12,13,Suboptimal foldings created.
Energy dot plot created.
10	9	24	14,15,16,17,18,19,9	
Structure plots generated.
All done.
20,21,22,23,13	18	24,25,26,27, 
13	15	11

19	Suboptimal foldings created.
Energy dot plot created.
10	170,180,Save file created using nafold.
Minimum folding energy is -110.30 kcal/mol.
Energy increment is 5.51 kcal/mol.
H-num file created from plot file.
1,2,190,3,4,5,6,7,8,H-num file created from plot file.
1,200,9,10, 
2,3,4,5,6,7,8,9,10,3	210, 
7	220,230,1	240,11	Suboptimal foldings created.
Energy dot plot created.
10	Suboptimal foldings created.
Energy dot plot created.
10	250,260,270,280,290,300,8	4	310,320,330,340,350,360,20	12	1	370,380,390,1	400,410, 
End of Fill
Save file created using nafold.
Minimum folding energy is -137.20 kcal/mol.
Energy increment is 6.86 kcal/mol.
H-num file created from plot file.
1,2,3,4,9	5	5,6,7,8,9,10,11,2	12,13,14,15,13	2	2	16,17,18, 

Structure plots generated.
All done.
6	3	14	3	3	/home/jupyter/plant_microRNA_prediction
/home/jupyter/plant_microRNA_prediction/secondary_structure/mfold/AMWY020014981|4144-4566|201-223|+
Suboptimal foldings created.
Energy dot plot created.
10	mfold vers

In [61]:
'''
base = "secondary_structure/mfold/"
for directory in glob.glob(f"{base}*"):    
    tag = directory[len(base):]
    ct_files = glob.glob(f'{directory}/*.ct')        
    try:
        ct_files.remove(f'{base}{tag}/SEQ.ct')
    except:
        print(directory)
        print(ct_files)
        print("*****************")
    for file in ct_files:        
        shutil.copy(file, './1.ct')
        #dot = ct2dot_bracket('./1.ct')
        #dot = dot.split('\n')
        #with open('./2.ct', 'w') as stream:
            #stream.write(bracket_to_ct(tag, dot[0] , dot[1] , "(0)"))        
        #ct1 = '\n'.join(reformatCT('./1.ct').split('\n')[1:])
        #ct2 = '\n'.join(reformatCT('./2.ct').split('\n')[1:])
        #if(ct1 != ct2):
            #print(file)
        ct = reformatCT('./1.ct')
        [nucleotide, index, values] = get_ct_data(ct)        
        #print(is_nested( index,  values))
        if(not is_nested( index,  values)):
            print("************")             
'''

## Mxfold2

In [57]:
#!wget https://github.com/keio-bioinformatics/mxfold2/releases/download/v0.1.1/mxfold2-0.1.1.tar.gz
#!pip3 install mxfold2-0.1.1.tar.gz
#!rm mxfold2-0.1.1.tar.gz

In [58]:
!mxfold2 predict ./extended.txt > secondary_structure/mxfold2_result.txt

>AMWY02059828.1:2832-3256(+)
AAAGAATCAGCAATGGAAAAATAACCGGTTCTTAATTCAGcataacaaattattcaattataatatagcTGTAAAAGAAATCTAAGTCTATTTGATATAGATCGGAATTTacgcaaattaaaaatttccaaataaGCAGTTCCGACCTGAGATCTGAACCGAAAACGCAAGATCCATCTAAACTCTCACCTCGGTCTCCGATTCAGTTGATGCAAGGCGGGATCCAATTCGCCTTTTCATTCAATTACATTCACCAATAACAGCTCGCCATCtggcttttaataaaaagttgcCAATCGGTTCCCGACCTGCACCAAGCGAATTAGAGACCGCCGGTAACTGAATCATTCTACATTAATCCCCGACTCCTCCTTTTACACATAGCAACTTCGCCCAAGAagactaaaaagaaaaggaagctAAC
........................((((...........((..........................))..........................................................................................................................(((...(((...............((((..........)))).................................(((....))).....................)))...)))..........................))))........................................................................................ (35.0)
>AMWY02004761.1:1853-2277(+)
actaataatgCATGGCCATATATATCAAATCTACCATATgccatttaataattttccttttttcttcttctttctttttctctct

>AMWY02002487.1:2516-2939(+)
tctctcttctttttcttccttaacTTTTCATCCAGCTTCAACCTCCATTTAGATCaaagttattgaatttttttttcatcttatttatgtaaatatatattgtttccTGCGGAAACGAATCCATGAACAACAGTCAATCAGTCATTGTTTGCTGATGCAGCGTCATCAAGATTCGCATGCTGATGGGTCGAGCAAAGCAGTGAGAATCTTGATGATGCTGCATCGGCCATAATTGACTATAtctcgtcatcatcatcatcatcatccagtTTCAACCTCCATGTAAATCaagttattgaattatttggtAAATAGATACTGATTCCCGCAGAATTGAATCAATGAACAACAGTCAATCAGTCATTGTTTGCTGATGCAGCATCATCAAGATTCACATGCGAATGGGTCGAACCAAAGCAGTGA
............................................................................................................((((....(....)............................((....)).(((...........))).........((.((.((...((..................)).))..)).)).................................................................................................))))........................................((....))....................((....((......))...))..... (30.2)
>AMWY02089812.1:0-419(-)
TCtgtctatatttattttcttctcattcACTGTAGTAATTTAAGCCTATACAGTTCTGAGTTGACCNatttctttatataaagtTNATTTC

>AMWY02001968.1:1243-1665(+)
CTACCTAAACTCCATGCATGGCTCGTGCTAGCTTTctggtttcttcttttttctttaagggcttattataaatttgcaGCAAGCCTAAACCCTTCTTAATTTCAAGATCTCTCTTcatttgattctttctttctttttctagggattcttcttcttcttcttcttcttgtttgctGCTGGTGTATGTTGGTTTGAGAGATTGAAGCTGCCAGCATGATCTGGTAATATGGAAcctaatattatacatatacatatctatatctatatatatagatagatagttttagatttactctttaattatattaattcctCCACCTATAGTTTTAGAGttactctttaattatattaattcttccACCTCTTTGTATAGATAGATAGAAACATATAGAAGGTCTTAGATTTCCTTTGCTTTTAGATCC
...((....(.((..((...((....))...............................(((....................)))..........................................................................................)).)).)......))........................................................................................................................................................................................................................................ (27.0)
>AMWY02039981.1:1214-1636(-)
atttatgttttctattttataattaaaaaataaaaaaaaaataataaggtaatctctctctctttttcttttatattcatatgaGGTCT

In [143]:
df = fasta_to_df('secondary_structure/mxfold2_result.txt')
df = df.apply(lambda row: bracket_row(row) , axis=1)
df.head(2)

,tag,data,bracket
0,AMWY02059828.1:2832-3256(+),AAAGAATCAGCAATGGAAAAATAACCGGTTCTTAATTCAGcataac...,........................((((...........((........
1,AMWY02004761.1:1853-2277(+),actaataatgCATGGCCATATATATCAAATCTACCATATgccattt...,.................................................


In [145]:
base = "./secondary_structure/mxfold2/"
!rm -r {base}
!mkdir -p {base}
for index, row in df.iterrows():    
    if(not os.path.exists(base + reformat(row['tag']))):
        os.makedirs(base + reformat(row['tag']))        
    tag = reformat(row['tag'])
    with open(base + f"{tag}/{tag}.ct",'w') as file:
        bracket = row['bracket'].split(' ')[0]
        deltaG = row['bracket'].split(' ')[1]
        ct = bracket_to_ct(row['tag'], row['data'], bracket, deltaG)
        file.write(ct)    

## SPOT-RNA

In [8]:
#!git clone https://github.com/jaswindersingh2/SPOT-RNA.git
#%cd SPOT-RNA
#!wget 'https://www.dropbox.com/s/dsrcf460nbjqpxa/SPOT-RNA-models.tar.gz' || wget -O SPOT-RNA-models.tar.gz 'https://app.nihaocloud.com/f/fbf3315a91d542c0bdc2/?dl=1'
#!tar -xvzf SPOT-RNA-models.tar.gz && rm SPOT-RNA-models.tar.gz
#!sudo apt-get install python3.6
#!python3.6 -m pip install tensorflow==1.14.0 # or for gpu: tensorflow-gpu==1.14.0
#! python3.6 -m pip install -r requirements.txt

In [23]:
base = "./secondary_structure/spot_rna/"
!rm -r {base}
!mkdir -p {base}

In [29]:
!python3.6 ./SPOT-RNA/SPOT-RNA.py  --inputs ./extended.txt  --outputs '{base}'  --cpu 32 --plots True

>> Opening FASTA file...
>> Converting FASTA file from multiline to single line and writing to file.
>> Done!

Preparing tfr records file for SPOT-RNA:
100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.21s/it]

Predicting for SPOT-RNA model 0
100%|█████████████████████████████████████████████| 1/1 [00:05<00:00,  5.23s/it]

Predicting for SPOT-RNA model 1
100%|█████████████████████████████████████████████| 1/1 [00:09<00:00,  9.51s/it]

Predicting for SPOT-RNA model 2
100%|█████████████████████████████████████████████| 1/1 [00:13<00:00, 13.10s/it]

Predicting for SPOT-RNA model 3
100%|█████████████████████████████████████████████| 1/1 [00:16<00:00, 16.44s/it]

Predicting for SPOT-RNA model 4
100%|█████████████████████████████████████████████| 1/1 [00:18<00:00, 18.18s/it]

Post Processing and Saving Output

Finished!

Processsing Time 203.32813096046448 seconds


In [48]:
!rm {base}/*.bpseq
!rm {base}/*.prob
for file in glob.glob(f"{base}*.ct"):    
    f = file[len(base):-3] # .ct        
    f = reformat(f)        
    if(not os.path.exists(base + f)):
        os.makedirs(base + f)  
    header = reformatCT(file).split("\n")[0]    
    with open(f"{base}{f}.dot", 'w') as stream:        
        stream.write(ct2dot_bracket(file))
    !RNAeval "{base}{f}.dot" -T 20 -v 
    #shutil.move(file, f"{base}{f}/{f}.ct")    

rm: cannot remove './RNA_secondary_structure/spot_rna//*.bpseq': No such file or directory
rm: cannot remove './RNA_secondary_structure/spot_rna//*.prob': No such file or directory
External loop                           :  -371
Interior loop (  3, 32) AU; (  4, 31) GC:  -256
Interior loop (  4, 31) GC; (  5, 30) AU:  -294
Interior loop (  5, 30) AU; (  6, 29) AU:  -122
Hairpin  loop (  6, 29) AU              :   716
Interior loop ( 38, 71) CG; ( 39, 70) AU:  -255
Interior loop ( 39, 70) AU; ( 40, 69) GC:  -256
Interior loop ( 40, 69) GC; ( 41, 68) CG:  -403
Interior loop ( 41, 68) CG; ( 49, 64) AU:   618
Interior loop ( 49, 64) AU; ( 50, 63) UA:  -155
Interior loop ( 50, 63) UA; ( 51, 62) UA:  -122
Interior loop ( 51, 62) UA; ( 52, 61) AU:  -165
Hairpin  loop ( 52, 61) AU              :   502
Interior loop ( 87,101) UA; ( 88,100) CG:  -294
Interior loop ( 88,100) CG; ( 89, 99) UA:  -256
Interior loop ( 89, 99) UA; ( 90, 98) AU:  -165
Hairpin  loop ( 90, 98) AU              :   573
Int

In [68]:
df = fasta_to_df('./secondary_structure/spot_rna/AMWY02059828.1:2832-3256(+).dot')
df = df.apply(lambda row: bracket_row(row) , axis=1)
bracket = df['bracket'][0].split(' ')[0]
ct = bracket_to_ct(df['tag'][0], df['data'][0], bracket, "(0)")
print(ct)

   424 dG =       0.0 AMWY02059828.1:2832-3256(+)
     1 A      0      2      0       1
     2 A      1      3      0       2
     3 A      2      4     32       3
     4 G      3      5     31       4
     5 A      4      6     30       5
     6 A      5      7     29       6
     7 U      6      8      0       7
     8 C      7      9      0       8
     9 A      8     10      0       9
    10 G      9     11      0      10
    11 C     10     12      0      11
    12 A     11     13      0      12
    13 A     12     14      0      13
    14 U     13     15      0      14
    15 G     14     16      0      15
    16 G     15     17      0      16
    17 A     16     18      0      17
    18 A     17     19      0      18
    19 A     18     20      0      19
    20 A     19     21      0      20
    21 A     20     22      0      21
    22 U     21     23      0      22
    23 A     22     24      0      23
    24 A     23     25      0      24
    25 C     24     26      0      25


## Vienna package

In [92]:
#!wget https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_20_04/viennarna_2.4.18-1_amd64.deb -O viennarna.deb
#!sudo dpkg -i ./viennarna.deb
#!sudo apt-get -f install
#!rm viennarna.deb

In [131]:
base = "./secondary_structure/viennarna/"
!rm -r {base}
!rm ./secondary_structure/viennarna_result.txt
!mkdir -p {base}

rm: cannot remove './secondary_structure/viennarna_result.txt': No such file or directory


In [132]:
%cd {base}
!RNAfold --jobs=0 --infile ../../Temp/extended_modified.txt  --noPS -T 20 > ../viennarna_result.txt
%cd ../../

/home/jupyter/plant_microRNA_prediction/secondary_structure/viennarna
/home/jupyter/plant_microRNA_prediction


In [95]:
df = fasta_to_df('secondary_structure/viennarna_result.txt')
df = df.apply(lambda row: bracket_row(row) , axis=1)
print(df.shape)
df.head(2)

(21225, 3)


,tag,data,bracket
0,AMWY02000003.1:4084-4506|200-222(+),auacaauugucacauaguuuacauuauuaauuuccgcuuaauuuau...,....((((((...))))))................((((((...((...
1,AMWY02000003.1:4391-4810|200-219(+),uucuaauugcaaauuuauguuauauuuuuaaauagaaagggagauu...,.....((.(((......))).))......(((((((.(((((((.....


In [125]:
for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)      
    with open(base + f"{tag}/{tag}.ct",'w') as file:
        bracket = row['bracket'].split(' ')[0]
        deltaG = row['bracket'].split(' ')[1]
        ct = bracket_to_ct(row['tag'], row['data'], bracket, deltaG, False)
        file.write(ct)    

In [126]:
import glob
for file in glob.glob(f"{base}*.ps"):    
    f = file[len(base):-6] # _ss.ps 
    f = reformat(f)        
    shutil.move(file, f"{base}{f}/{f}.ps")    

## ContraFold

In [39]:
#!wget http://contra.stanford.edu/contrafold/contrafold_v2_02.tar.gz
#!tar -xvzf contrafold_v2_02.tar.gz && rm contrafold_v2_02.tar.gz
#%cd contrafold/src
#!make clean
#!make 
# to file must changed to be complieable # utility.hpp and optimization.c++ files

In [42]:
counter = 0
base = "./secondary_structure/contrafold/"
!rm -r {base}
!mkdir -p {base}
df = fasta_to_df('./Temp/extended.txt')

for index, row in df.iterrows():    
    tag = reformat(row['tag'])
    if(not os.path.exists(base + tag)):
        os.makedirs(base + tag)            
    with open(base + f"{tag}/{tag}.FASTA",'w') as file:
        file.write(f">{row['tag']}\n{row['data']}")
    counter += 1    
    if(counter >= 10):
        break

In [43]:
def run_contrafold(tag):
    tag = reformat(tag)    
    %cd contrafold/src
    !./contrafold predict ../..{base[1:]}{tag}/{tag}.FASTA > ../..{base[1:]}{tag}/{tag}.dot
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'r') as file:
        text = file.read()
    text = [l for l in text.split("\n") if l[:len(">structure")] != ">structure"]    
    header = text[0]
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'w') as file:
        file.write('\n'.join(text[1:]))    
    !RNAeval  ../..{base[1:]}{tag}/{tag}.dot -T 20 > ../..{base[1:]}{tag}/{tag}.dotdg    
    with open(f"../..{base[1:]}{tag}/{tag}.dotdg", 'r') as file:
        text = file.read()
    with open(f"../..{base[1:]}{tag}/{tag}.dot", 'w') as file:
        file.write(header + "\n" + text)    
    
    df = fasta_to_df(f'../..{base[1:]}{tag}/{tag}.dot')
    df = df.apply(lambda row: bracket_row(row) , axis=1)        
    tag = reformat(df['tag'][0])
    with open(f'../..{base[1:]}{tag}/{tag}.ct','w') as file:
        bracket = df['bracket'][0].split(' ')[0]        
        deltaG = df['bracket'][0].split(' ')[1]
        ct = bracket_to_ct(df['tag'][0], df['data'][0], bracket, deltaG, False)
        file.write(ct)    
    #!rm ../..{base[1:]}{tag}/{tag}.dot
    #!rm ../..{base[1:]}{tag}/{tag}.dotdg
    !rm ../..{base[1:]}{tag}/{tag}.FASTA
    %cd ../../        

if __name__ == '__main__':        
    pool = mp.Pool(mp.cpu_count() - 1)  
    pool.map(run_contrafold, df['tag'].iloc[:10])

In [56]:
s = 'CUCCCCUUGUCUACCAUCCCCAACUAGCGAGAGAGACAUUACCUACCUGAAUAGAAGAUCUCUCUCGAGCUCUCGagcucucucuuuuucuauaUCUCUGUCUCUUUGUGUCUCUGGAGCUUGUACUAACAUUAAUAUCGUGCACCAGCAGCAGUUGAAGCUGCCAGCAUGAUCUAAACUUCCUUCUCUGUAAAGGAUAGAUCGGAUCAUGUGGUAGCUUCACCUGUUGAUGGGAUCACGAAAGCGCCCCUCUUACUACUCUACAUUAAUUCUUUCUCGUUAUACAACCUCCCAGUAAGCAUGCUUUCAAAACCAACUUGAGuaaguuaauuuguuuagcuuuuguuuuuggcucuuccuuuacuuuaaauuuucucaucuggguuuuuguuauauauauguacuguuuuauauauguauuccu'
d = '............................((((((((..(...(((......))).)..))))))))(((((....)))))...................................((((.((((...(((.......(((((..(((.((((((.((((((((((.(((((((((.(.(((((((.......))))).)).).))))))))))))))))))).)))))).)))...))))).....................................)))..)))).))))....((((()))))..((((((....((((.(((((((.....)))(((.........)))................)))).))))....))))))....(((((((((((......)))))))))))....'
print(s[300],s[301])
print(d[300],d[301])

A U
( )


In [55]:
'''path = 'secondary_structure/contrafold/AMWY020333941_469-893_-_/AMWY020333941_469-893_-_.dot'
!RNAeval  {path} -T 20 -v'''; 

External loop                           :  -364
Interior loop ( 29, 66) GC; ( 30, 65) AU:  -294
Interior loop ( 30, 65) AU; ( 31, 64) GC:  -256
Interior loop ( 31, 64) GC; ( 32, 63) AU:  -294
Interior loop ( 32, 63) AU; ( 33, 62) GC:  -256
Interior loop ( 33, 62) GC; ( 34, 61) AU:  -294
Interior loop ( 34, 61) AU; ( 35, 60) GC:  -256
Interior loop ( 35, 60) GC; ( 36, 59) AU:  -294
Interior loop ( 36, 59) AU; ( 39, 56) UA:   103
Interior loop ( 39, 56) UA; ( 43, 54) CG:   305
Interior loop ( 43, 54) CG; ( 44, 53) UA:  -256
Interior loop ( 44, 53) UA; ( 45, 52) AU:  -165
Hairpin  loop ( 45, 52) AU              :   493
Interior loop ( 67, 80) GC; ( 68, 79) AU:  -294
Interior loop ( 68, 79) AU; ( 69, 78) GC:  -256
Interior loop ( 69, 78) GC; ( 70, 77) CG:  -403
Interior loop ( 70, 77) CG; ( 71, 76) UA:  -256
Hairpin  loop ( 71, 76) UA              :   533
Interior loop (116,292) GC; (117,291) GC:  -385
Interior loop (117,291) GC; (118,290) AU:  -294
Interior loop (118,290) AU; (119,289) GC

# Visualization

In [25]:
#https://github.com/ViennaRNA/forna
#http://varna.lri.fr/

# CT Analizer

In [14]:
# only select those not ran before
base = "./secondary_structure/mfold/"
df = fasta_to_df('./Temp/extended_modified.txt')

index_list =[]
for index, row in df.iterrows():    
    tag = reformat(row['tag'])    
    if(len(glob.glob(f'{base + tag}/*.ct')) != 0):
        index_list.append(index)
df = df.iloc[index_list,:]
print(df.shape)

(1000, 2)


In [15]:
tag = df.iloc[3,:]['tag']
print(f'{base}{reformat(tag)}/SEQ_1.ct')
ct = reformatCT(f'{base}{reformat(tag)}/SEQ_1.ct')
data = tag.split('|')
hit_start = int(data[2].split('-')[0]) - 1 
hit_end = int(data[2].split('-')[1])
sign = data[3]

ct_head = ct.split('\n')[0]

if("dG = " in ct_head):    
    dG_patter = "dG = " 
elif("dG= " in ct_head):    
    dG_patter = "dG= "
elif("dG=" in ct_head):    
    dG_patter = "dG="
elif("dG =" in ct_head):    
    dG_patter = "dG ="
else:
    print('there is no dG')

dG = float(ct_head.split(dG_patter)[-1].split(' ')[0])
[nucleotide, index, values] = get_ct_data(ct)

./secondary_structure/mfold/AMWY020000141|194-614|201-221|+/SEQ_1.ct


In [16]:
hit_seq = ''.join(nucleotide[hit_start:hit_end])
hit_range = index[hit_start:hit_end]
hit_len = len(hit_range)
inc_srange = values[hit_start:hit_end] # Incomplete_Star_range

In [17]:
if(sum(inc_srange == 0) == hit_len):
    complementarity_in_hit_region = "no"
elif(sum(inc_srange != 0) == hit_len):
    complementarity_in_hit_region = "fully_connected"
else:    
    complementarity_in_hit_region = "yes"
print(complementarity_in_hit_region)

yes


In [18]:
# Hit_self_complementarity
hit_self_complementarity = not ((inc_srange <= hit_start) | (inc_srange > hit_end)).all()
hit_self_complementarity

False

In [19]:
nonzero_data_srange  = inc_srange[inc_srange!=0]
istar_min = nonzero_data_srange.min()
istar_max = nonzero_data_srange.max()
if(hit_self_complementarity):
    istar_min = np.nan
    istar_max = np.nan
print(istar_min)
print(istar_max)

369
388


In [20]:
if(hit_self_complementarity):
    continuous_pairing = "undifined"
else:
    if(hit_end < istar_min and (hit_start+1) > istar_min):
        continuous_pairing = "no"
    else:
        continuous_pairing = "yes"
print(continuous_pairing)

yes


In [29]:
### miR type
if(continuous_pairing == "yes" and complementarity_in_hit_region != "no" and hit_self_complementarity == "no"):
    if( hit_end < istar_min):
        mir_type = "5p" 
    if( hit_start > istar_max):
        mir_type = "3p"     
else:
    if(continuous_pairing == "no" and hit_self_complementarity == "yes"):
        mir_type = "discontinuous star strand and hit self complementarity"
    elif(continuous_pairing == "no"):
        mir_type = "discontinuous star strand"
    elif(hit_self_complementarity == "yes"):
        mir_type = "hit self complementarity"

if(complementarity_in_hit_region == "no"):
    mir_type = "no complementarity in hit region"    
print(mir_type)

5p


In [22]:
c = 0
i = hit_end - 3 - c
while(values[i] == 0 and i >= hit_start):
    c += 1
    i = hit_end - 3 - c
    
if(i >= hit_start):
    star_start = values[i] - c
    if(star_start < 1):
        print("star_end error!")
else:
    star_start = np.nan
print(star_start, c)

371 0


In [23]:
if(hit_start - 2 >= 0 ):
    a = 0    
else:
    a = abs(hit_start - 2)
    
i = hit_start - 2 + a
while(values[i] == 0 and i <= hit_end):
    a += 1
    i = hit_start - 2 + a
    
if(i <= hit_end):
    star_end = values[i] + a
else:
    star_end = np.nan
print(star_end, a)

390 0


In [24]:
star_range = index[star_start - 1:star_end]
star_seq = ''.join(nucleotide[star_start - 1:star_end])

In [25]:
if(mir_type == '5p'):
    num_of_linking_residues = str(star_start - hit_end - 1)
elif(mir_type == '3p'):
    num_of_linking_residues = str(hit_start - star_end)
elif(mir_type == "discontinuous star strand"):
    num_of_linking_residues = "discontinuous star strand"
elif(mir_type == "no complementarity in hit region"):
    print('error')
print(num_of_linking_residues)

149


In [203]:
# self Hit_self_complementarity
star_branching = not ((values[star_range-1] < star_start) | (values[star_range-1] > star_end)).all()
star_branching

False

In [71]:
def getBOI_5p():
    # first calc latest non zero value
    for i in range(hit_end-1, 0, -1):
        if(values[i] != 0):
            last_v = values[i]
            last_i = i
            place = i
            break            
            
    for i in range(place-1, 0, -1):        
        v = values[i]
        if(v == 0):
            continue
        if(v < last_v):
            return [last_i + 1, last_v]                                                
        
        if((v - last_v) >= 3):
            s1 = set(range(last_v+1, v))
            s2 = set([values[i-1] for i in range(last_v+1, v)])
            if(len(s1.intersection(s2)) > 0):
                return [last_i + 1, last_v]    
        last_v = v            
        last_i = i            
    for i in range(0,hit_end):
        if(values[i] != 0 ):
            return [i + 1, values[i]]    
    print("Error")
                
def getBOI_3p():
    # first calc latest non zero value
    for i in range(hit_start, len(values)):    
        if(values[i] != 0):
            last_v = values[i]
            last_i = i
            place = i
            break            
            
    for i in range(place + 1, len(values)):
        v = values[i]
        if(v == 0):
            continue
        if(v > last_v):
            return [last_i + 1, last_v]                                                
        
        if((last_v - v) >= 3):
            s1 = set(range(v+1, last_v))
            s2 = set([values[i-1] for i in range(v+1, last_v)])
            if(len(s1.intersection(s2)) > 0):
                return [last_i + 1, last_v]    
        last_v = v            
        last_i = i            
    for i in range(hit_start, len(values)):
        if(values[i] != 0 ):
            return [i + 1, values[i]]    
    print("Error")            
    
    
#BOI, Branch of interest
if(mir_type not in ['3p','5p']):
    pass
else:
    if(mir_type == '5p'):
        [boi_start, boi_end] = getBOI_5p()
    if(mir_type == '3p'):
        [boi_start, boi_end] = getBOI_3p()    

# BLASTX or DIMOND

# DIAMOND

https://github.com/bbuchfink/diamond

In [ ]:
'''
import hashlib
import os
parallel = [os.path.join(dp, f) for dp, dn, filenames in os.walk("./PRNA_secondary_structure") for f in filenames ]
series = [os.path.join(dp, f) for dp, dn, filenames in os.walk("./SRNA_secondary_structure") for f in filenames ]
for i in range(len(parallel)):        
    md5_hash = hashlib.md5()
    with open(parallel[i],"rb") as file:        
        md5_hash.update(file.read())
        digest1 = md5_hash.hexdigest()                
    md5_hash = hashlib.md5()
    with open(series[i],"rb") as file:        
        md5_hash.update(file.read())
        digest2 = md5_hash.hexdigest()                
    if(digest1 != digest2):
        print(parallel[i])
        print(series[i])
        print("***********")     
'''